In [1]:
# Installing libraries required for this task

import sys
import torch
import transformers
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sklearn.metrics
import tqdm
import matplotlib.pyplot as plt
import nltk

# vector LLM toolkit 
import lingua

# Print version information - check you are using correct environment
print("Python version: " + sys.version)
print("PyTorch version: " + torch.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
PyTorch version: 1.13.1+cu117
Transformers version: 4.26.1


# Getting Started

Next, We will be starting with connecting to the Lingua service through which we can connect to the large language model, OPT-175B. We will also be checking how many models are available to us.

In [2]:
# Establish a client connection to the Lingua service
client = lingua.Client(gateway_host="llm.cluster.local", gateway_port=3001)

In [3]:
# checking how many models are available for use
client.models

['OPT-175B', 'OPT-6.7B']

In [4]:
# checking how many model instances are active
client.model_instances

[{'id': 'b11f3264-9c03-4114-9d56-d39a0fa63640',
  'name': 'OPT-175B',
  'state': 'ACTIVE'},
 {'id': 'c5ea8da7-3384-4c7b-b47f-95ed1a485897',
  'name': 'OPT-6.7B',
  'state': 'ACTIVE'}]

In [5]:
# For this notebook, we will be focusing on OPT-175B

model = client.load_model("OPT-175B")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

### Loading model and tokenizer

Need to instantiate a tokenizer to obtain appropriate token indices for our labels.
NOTE: All OPT models, regardless of size, used the same tokenizing. However, if you want to use a different type of model, a different tokenizer may be needed.

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [7]:
class CustomDataset(Dataset):
    """
    A class for the dataset
    ...
    
    Attributes
    ----------
    df : pandas dataframe
        Dataset in the format of a pandas dataframe. 
        Ensure it has columns named sentence_with_full_prompt and aspect_term_polarity

    Methods
    -------
    """
    def __init__(self, df):
        self.df = df

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text_prompt = row['sentence_with_full_prompt']
        polarity = row['aspect_term_polarity']
        text = row['text']
#         if index == 100:
        return text, text_prompt, polarity

    def __len__(self):
        return len(self.df)
    
def flatten(l):
    '''
    Flattens list of lists to list
    
    Parameters
    ----------
    l (list): 
        list of lists

    Returns
    ----------
    list (list): 
        Flattened list
    '''
    return [item for sublist in l for item in sublist]

# Stylistic function
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

### Some prompts

#### Prompt Setup for Input
* `df['sentence_with_prompt'] = 'Sentence: ' + df['text'] + ' ' + 'Sentiment on ' + df['aspect_term'] + ' is'`


* `df['sentence_with_prompt'] = 'Sentence: ' + df['text'] + ' ' + 'Sentiment on ' + df['aspect_term'] + ' is positive or negative? It is'`


* `df['sentence_with_prompt'] = 'Answer the question using the sentence provided. \nQuestion: What is the sentiment on ' + df['aspect_term'] + ' - positive, negative, or neutral?' + '\nSentence: ' + df['text'] + '\nAnswer:'`


* `df['sentence_with_prompt'] = 'Sentence: ' + df['text'] + ' ' + 'The sentiment associated with ' + df['aspect_term'] + ' is'`


* `df['sentence_with_prompt'] = '\nSentence: ' + df['text'] + ' ' + '\nQuestion: What is the sentiment on ' + df['aspect_term'] + '? \nAnswer:'`

#### Few-shot Examples
* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. Sentiment on Albert Einstein is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. Sentiment on rasmalai is negative. '`


* `promptStarting = 'Sentence: Their pizza was good, but the service was bad. Sentiment on pizza is positive. \nSentence: I charge it at night and skip taking the cord with me because its''s too heavy. Sentiment on cord is negative. \nSentence: My suggestion is to eat family style because you''ll want to try the other dishes. Sentiment on dishes is neutral. '`


* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. Sentiment on Albert Einstein is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. Sentiment on rasmalai is negative. \nSentence: I had my software installed and ready to go, but the system crashed. Sentiment on software is neutral. '`


* `promptStarting = 'Sentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. Sentiment on rasmalai is negative. \nSentence: I had my software installed and ready to go, but the system crashed. Sentiment on software is neutral. \nSentence: Albert Einstein was one of the greatest intellects of his time. Sentiment on Albert Einstein is positive.'`


* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. Sentiment on Albert Einstein is positive or negative? It is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. Sentiment on rasmalai is positive or negative? It is negative. '`


* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. The sentiment associated with Albert Einstein is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. The sentiment associated with rasmalai is negative. '`


* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. The sentiment associated with Albert Einstein is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. The sentiment associated with rasmalai is negative. \nSentence: I had my software installed and ready to go, but the system crashed. The sentiment associated with software is neutral. '`


* `promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. \nQuestion: What is the sentiment on Albert Einstein? \nAnswer: postive \n\nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. \nQuestion: What is the sentiment on rasmalai? \nAnswer: negative '`

In this notebook we will be showing accuracy on three different approaches, few-shot, zero-shot and zero-shot with Sentiment Classifier.

* few-shot approach [`few-shot`] (i.e. give some example sentences for the model to determine what should come next for the input sentence) 
* zero-shot approach [`zero-shot`] (i.e. give the model the input sentence and ask for the sentiment) 

In [8]:
generation_type = "few-shot" # other option here "zero-shot"

In [11]:
# This csv file contains customer reviews of laptops collected in 2014 with size of around 469
path="/scratch/ssd004/scratch/akshita/vector_intern23/PromptEngineering/src/reference_implementations/prompting_vector_llms/llm_prompting_examples/resources/absa_datasets/"
datasets_list = [path+'Laptops_Test_Gold.csv'] #Here you can add more datasets like 'Laptop_Train_v2.csv', 'Restaurants_Train_v2.csv', and 'Restaurants_Test_Gold.csv'

In [ ]:
for d in tqdm.notebook.tqdm(range(len(datasets_list))):
    df = pd.read_csv(datasets_list[d])
    
    print('----------------------------------------------------------------')
    df.info()
    print()

    # Delete any rows with null values
    df = df.dropna(axis=0, how='any', subset=['aspect_term', 'aspect_term_polarity'])
    
    # Set the prompt format for the input sentence
    df['sentence_with_prompt'] = 'Sentence: ' + df['text'] + ' ' + 'Sentiment on ' + df['aspect_term'] + ' is positive, negative, or neutral? It is'
        
    df = df.loc[df['aspect_term_polarity'].str.contains('positive') | df['aspect_term_polarity'].str.contains('negative')]
    promptStarting = 'Sentence: Albert Einstein was one of the greatest intellects of his time. Sentiment on Albert Einstein is positive, negative, or neutral? It is positive. \nSentence: The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable. Sentiment on rasmalai is positive, negative, or neutral? It is negative. \nSentence: I had my software installed and ready to go, but the system crashed. Sentiment on software is positive, negative, or neutral? It is neutral.'
    ## zero-shot
    
    # for few-shot, we give more context to the model to improve the model generalizability. 
    if generation_type == 'few-shot': 
        df['sentence_with_full_prompt'] = promptStarting + '\n' + df['sentence_with_prompt']
    elif generation_type == 'zero-shot':
        df['sentence_with_full_prompt'] = df['sentence_with_prompt']
    else: 
        print('Do you want few-shot or zero-shot Set the generation_type correctly')    
    
    df.info()
    
    # Called the dataloader with custom created dataset as input
    data = CustomDataset(df)
    dataloader = DataLoader(data, batch_size=31)
    
    # initialize predictions and labels
    predictions = []
    labels = []

    ## parsing through the model alongth with vector hosted OPT-175B
    for text, text_prompt, polarity in tqdm.notebook.tqdm(dataloader):
        gen_text = model.generate(text_prompt, {"max_tokens": 1, "top_k": 4, "top_p": 3, "rep_penalty": 1.0, "temperature": 1.0}).generation["tokens"]
        first_predicted_tokens = [tokens[0].strip().lower() for tokens in gen_text]
        sent = []
        for i in range(len(first_predicted_tokens)):           
            predictions.append(first_predicted_tokens[i])
        labels.append(list(polarity))
        
    labels = flatten(labels)
    # The labels associated with the dataset
    labels_order = ["positive", "negative"]

    cm = sklearn.metrics.confusion_matrix(np.array(labels), np.array(predictions), labels=labels_order)
    
    FP = cm.sum(axis=0) - np.diag(cm)
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)

    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = 2 * (precision * recall) / (precision + recall)
    print(f"Prediction Accuracy: {TP.sum()/(cm.sum())}")
    
    print(f"Confusion Matrix with ordering {labels_order}")
    print(matrix)
    print("========================================================")
    for label_index, label_name in enumerate(labels_order):
        print(
            f"Label: {label_name}, F1: {f1[label_index]}, Precision: {recall[label_index]}, "
            f"Recall: {precision[label_index]}"
        )

  0%|          | 0/1 [00:00<?, ?it/s]

----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    800 non-null    object 
 1   text                  1032 non-null   object 
 2   aspect_term           654 non-null    object 
 3   aspect_term_polarity  654 non-null    object 
 4   aspect_term_from      654 non-null    float64
 5   aspect_term_to        654 non-null    float64
dtypes: float64(2), object(4)
memory usage: 48.5+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469 entries, 0 to 1031
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         314 non-null    object 
 1   text                       469 non-null    object 
 2   aspect_term                469 no

  0%|          | 0/16 [00:00<?, ?it/s]